In [ ]:
import os
from typing import List, Optional
from abc import ABC, abstractmethod
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.tools import Tool
from langchain.tools.retriever import create_retriever_tool
from langchain.schema import Document
from dotenv import load_dotenv

load_dotenv()


In [ ]:
def summarize_documents(docs: list[Document]) -> str:

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    all_text = "\n".join([doc.page_content for doc in docs])

    prompt = f"Fasse diesen Text in 1-2 Sätzen zusammen:\n---\n{all_text}\n---"
    summary = llm.invoke(prompt)
    return summary.content.strip()

In [ ]:
class IVectorStoreObserver(ABC):
    """
    Ein Interface für alle Observer, die benachrichtigt werden möchten,
    sobald ein VectorStore neue Dokumente bekommt / aktualisiert wird.
    """
    @abstractmethod
    def on_vectorstore_update(self, manager: "SingleVectorStoreManager"):
        pass


In [ ]:
class SingleVectorStoreManager:
    def __init__(self, persist_dir: str):
        self.embedding_function = OpenAIEmbeddings()
        self.persist_dir = persist_dir

        collection_name = os.path.basename(persist_dir)
        self.vs = Chroma(
            collection_name=collection_name,
            embedding_function=self.embedding_function,
            persist_directory=self.persist_dir
        )

        self.description = "Dieser Vectorstore ist leer."

        self.observers: List[IVectorStoreObserver] = []

    def add_observer(self, observer: IVectorStoreObserver):
        self.observers.append(observer)

    def remove_observer(self, observer: IVectorStoreObserver):
        if observer in self.observers:
            self.observers.remove(observer)

    def notify_observers(self):
        for obs in self.observers:
            obs.on_vectorstore_update(self)

    def is_empty(self) -> bool:
        return (self.vs._collection.count() == 0)

    def create_retriever_tool(self, name: str, custom_description: Optional[str] = None) -> Tool:

        retriever = self.vs.as_retriever()
        desc = custom_description if custom_description else self.description
        if self.is_empty():
            desc += "\n(Hinweis: Dieser Vectorstore ist aktuell leer.)"

        tool = create_retriever_tool(
            retriever=retriever,
            name=name,
            description=desc
        )
        return tool

    def add_documents(self, docs: list[Document], update_description: bool = True):

        self.vs.add_documents(docs)
        if update_description:
            summary_text = summarize_documents(docs)
            if self.is_empty():
                pass
            self.description = (
                "Der Vectorstore enthält nun neue Dokumente. "
                f"Zusammenfassung: {summary_text}"
            )
        self.notify_observers()

In [ ]:
class LLMToolBinder(IVectorStoreObserver):

    def __init__(self, llm: ChatOpenAI, managers: List[SingleVectorStoreManager]):
        self.llm = llm
        self.managers = managers
        self.tools: List[Tool] = []
        self._bind_tools()

    def _bind_tools(self):

        new_tools = []
        for i, m in enumerate(self.managers, start=1):
            tool_name = f"retriever_store{i}"
            new_tools.append(m.create_retriever_tool(name=tool_name))
        self.tools = new_tools

        self.llm = self.llm.bind_tools(self.tools)

    def on_vectorstore_update(self, manager: SingleVectorStoreManager):
        print(f"[LLMToolBinder] VectorStore '{manager.persist_dir}' hat Update erhalten. Re-binde Tools.")
        self._bind_tools()

    def invoke_llm(self, user_query: str) -> str:
        from langchain_core.messages import HumanMessage, ToolMessage

        messages = [HumanMessage(content=user_query)]
        first_output = self.llm.invoke(messages)
        messages.append(first_output)

        if first_output.tool_calls:
            for tool_call in first_output.tool_calls:
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]

                found_tool = None
                for t in self.tools:
                    if t.name.lower() == tool_name.lower():
                        found_tool = t
                        break

                if not found_tool:
                    tool_result = f"ERROR: No tool named '{tool_name}'"
                else:
                    tool_result = found_tool.invoke(tool_args)

                messages.append(ToolMessage(content=tool_result, tool_call_id=tool_call["id"]))

            second_output = self.llm.invoke(messages)
            messages.append(second_output)
            return second_output.content
        else:
            return first_output.content

    def print_all_tool_descriptions(self):
        for tool in self.tools:
            print(f"Tool Name: {tool.name}")
            print(f"Description: {tool.description}")
            print("-" * 40)

In [ ]:
base_dir = "my_chroma_db"
os.makedirs(base_dir, exist_ok=True)

manager1 = SingleVectorStoreManager(os.path.join(base_dir, "store1"))
manager2 = SingleVectorStoreManager(os.path.join(base_dir, "store2"))
manager3 = SingleVectorStoreManager(os.path.join(base_dir, "store3"))


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

binder = LLMToolBinder(llm, [manager1, manager2, manager3])

manager1.add_observer(binder)
manager2.add_observer(binder)
manager3.add_observer(binder)

In [ ]:
binder.invoke_llm("Where is Lacarelli?")

In [ ]:
docs_store1 = [
    Document(
        page_content=(
            "Lacarelli is a charming family-run Italian restaurant nestled in the "
            "heart of Berlin. Its menu features authentic dishes like homemade "
            "ravioli, wood-fired pizzas, and creamy tiramisu. With friendly staff, "
            "rustic decor, and a cozy atmosphere, Lacarelli provides an inviting "
            "dining experience for lovers of Italian cuisine and fine wines daily."
        )
    )
]
manager1.add_documents(docs_store1, update_description=True)

In [ ]:
binder.print_all_tool_descriptions()

In [ ]:
binder.invoke_llm("Where is Lacarelli?")